In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

import tensorflow as tf

import yfinance as yf

2025-10-05 15:09:23.101212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-05 15:09:23.107647: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 15:09:23.561221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-05 15:09:25.994863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

# Loading the data

Analysis and holdings

In [2]:
earnings_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_estimate.csv')
earnings_history = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_history.csv')
eps_revisions = pd.read_csv('data/analysis_and_holdings/AAPL_eps_revisions.csv')
eps_trend = pd.read_csv('data/analysis_and_holdings/AAPL_eps_trend.csv')
growth_estimates = pd.read_csv('data/analysis_and_holdings/AAPL_growth_estimates.csv')
insider_purchases = pd.read_csv('data/analysis_and_holdings/AAPL_insider_purchases.csv')
insider_roster_holders = pd.read_csv('data/analysis_and_holdings/AAPL_insider_roster_holders.csv')
institutional_holders = pd.read_csv('data/analysis_and_holdings/AAPL_institutional_holders.csv')
major_holders = pd.read_csv('data/analysis_and_holdings/AAPL_major_holders.csv')
mutualfunds_holders = pd.read_csv('data/analysis_and_holdings/AAPL_mutualfund_holders.csv')
reccomendations = pd.read_csv('data/analysis_and_holdings/AAPL_recommendations.csv')
revenue_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_revenue_estimate.csv')
sustainability = pd.read_csv('data/analysis_and_holdings/AAPL_sustainability.csv')
upgrades_downgrades = pd.read_csv('data/analysis_and_holdings/AAPL_upgrades_downgrades.csv')

Financials

In [3]:
balance_sheet = pd.read_csv('data/financials/AAPL_balance_sheet.csv', index_col=0)
cashflow = pd.read_csv('data/financials/AAPL_cashflow.csv', index_col=0)
income_statement = pd.read_csv('data/financials/AAPL_income_statement.csv', index_col=0)

Info

In [4]:
actions = pd.read_csv('data/info/AAPL_actions.csv', index_col=0)
capital_gains = pd.read_csv('data/info/AAPL_capital_gains.csv', index_col=0)
dividends = pd.read_csv('data/info/AAPL_dividends.csv', index_col=0)
fast_info = pd.read_csv('data/info/AAPL_fast_info.csv', index_col=0)
history = pd.read_csv('data/info/AAPL_history.csv', index_col=0)
info = pd.read_csv('data/info/AAPL_info.csv')
splits = pd.read_csv('data/info/AAPL_splits.csv', index_col=0)

Price

In [5]:
ohlc = pd.read_csv('data/price/AAPL_price_volume.csv', index_col=0)

# Data exploration

Price

In [6]:
ohlc.tail()

,Close,High,Low,Open,Volume
Price,,,,,
2025-09-29,254.42999267578125,255.0,253.00999450683594,254.55999755859375,40127700
2025-09-30,254.6300048828125,255.9199981689453,253.11000061035156,254.86000061035156,37704300
2025-10-01,255.4499969482422,258.7900085449219,254.92999267578125,255.0399932861328,48713900
2025-10-02,257.1300048828125,258.17999267578125,254.14999389648438,256.5799865722656,42630200
2025-10-03,258.0199890136719,259.239990234375,253.9499969482422,254.6699981689453,49107000


In [7]:
ohlc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11296 entries, Ticker to 2025-10-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   11295 non-null  object
 1   High    11295 non-null  object
 2   Low     11295 non-null  object
 3   Open    11295 non-null  object
 4   Volume  11295 non-null  object
dtypes: object(5)
memory usage: 529.5+ KB


In [8]:
ohlc.dtypes

Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

In [9]:
numeric_cols = ['Open', 'High', 'Low', 'Close']

for col in numeric_cols:
    ohlc[col] = pd.to_numeric(ohlc[col], errors='coerce')


In [10]:
ohlc.describe(include='all')

,Close,High,Low,Open,Volume
count,11294.000000,11294.000000,11294.000000,11294.000000,11295
unique,NaN,NaN,NaN,NaN,10731
top,NaN,NaN,NaN,NaN,246400000
freq,NaN,NaN,NaN,NaN,7
mean,26.689600,26.958130,26.394175,26.666814,NaN
std,55.240040,55.781888,54.632914,55.179857,NaN
min,0.037681,0.038109,0.037681,0.038109,NaN
25%,0.245137,0.250706,0.240161,0.245127,NaN
50%,0.466639,0.474725,0.459261,0.466445,NaN
75%,20.927646,21.097477,20.743933,20.965368,NaN


In [11]:
ohlc.memory_usage()

Index     90368
Close     90368
High      90368
Low       90368
Open      90368
Volume    90368
dtype: int64

In [12]:
print(len(ohlc['Close'].unique()))

8282


In [13]:
ohlc['Volume'] = pd.to_numeric(ohlc['Volume'], errors='coerce')
ohlc['High'] = pd.to_numeric(ohlc['High'], errors='coerce')
ohlc['Low'] = pd.to_numeric(ohlc['Low'], errors='coerce')
ohlc['Open'] = pd.to_numeric(ohlc['Open'], errors='coerce')
ohlc['Close'] = pd.to_numeric(ohlc['Close'], errors='coerce')


all_time_low = ohlc['Low'].min()
all_time_low_date = ohlc['Low'].idxmin()
print(f'All time low: {all_time_low} on {all_time_low_date}')

all_time_high = ohlc['High'].max()
all_time_high_date = ohlc['High'].idxmax()
print(f'All time high: {all_time_high} on {all_time_high_date}')

all_time_low_vol = ohlc['Volume'].min()
all_time_low_vol_date = ohlc['Volume'].idxmin()
print(f'All time low volume: {all_time_low_vol} on {all_time_low_vol_date}')

all_time_high_vol = ohlc['Volume'].max()
all_time_high_vol_date = ohlc['Volume'].idxmax()
print(f'All time high volume: {all_time_high_vol} on {all_time_high_vol_date}')

All time low: 0.037681121379137 on 1982-07-08
All time high: 259.239990234375 on 2025-10-03
All time low volume: 0.0 on 1981-08-10
All time high volume: 7421640800.0 on 2000-09-29


Balance Sheet

In [14]:
balance_sheet.tail()

,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Cash Cash Equivalents And Short Term Investments,6.517100e+10,6.155500e+10,4.830400e+10,6.263900e+10
Other Short Term Investments,3.522800e+10,3.159000e+10,2.465800e+10,2.769900e+10
Cash And Cash Equivalents,2.994300e+10,2.996500e+10,2.364600e+10,3.494000e+10
Cash Equivalents,2.744000e+09,1.606000e+09,5.100000e+09,1.763500e+10
Cash Financial,2.719900e+10,2.835900e+10,1.854600e+10,1.730500e+10


# Data preprocessing

In [15]:
data = ohlc.copy()

# Feature engineering

Adding percentage change of the closing price

In [16]:
data['Return'] = data['Close'].pct_change()

Adding volatility indicators

In [17]:
data['Volatility_20'] = data['Close'].rolling(window=20).std()
data['Volatility_50'] = data['Close'].rolling(window=50).std()

Adding ATR (Average True Range) — captures volatility from high/low ranges

In [18]:
high_low = data['High'] - data['Low']
high_close = (data['High'] - data['Close'].shift()).abs()
low_close = (data['Low'] - data['Close'].shift()).abs()

data['TR'] = high_low.combine(high_close, max).combine(low_close, max)
data['ATR_14'] = data['TR'].rolling(window=14).mean()

Adding moving averages

In [19]:
windows = [5, 20, 50, 200]

for w in windows:
    data[f'SMA_{w}'] = data['Close'].rolling(window=w).mean()
    data[f'EMA_{w}'] = data['Close'].ewm(span=w, adjust=False).mean()

Momentum indicators

In [20]:
# RSI (Relative Strength Index)
delta = data['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# MACD (Moving Average Convergence Divergence)
ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()

data['MACD'] = ema_12 - ema_26
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Stochastic Oscillator
low_14 = data['Low'].rolling(14).min()
high_14 = data['High'].rolling(14).max()

data['Stochastic'] = 100 * (data['Close'] - low_14) / (high_14 - low_14)

# ADX (Average Directional Index)
high = data['High']
low = data['Low']
close = data['Close']

plus_dm = high.diff()
minus_dm = low.diff().abs()
plus_dm[plus_dm < 0] = 0
minus_dm[minus_dm < 0] = 0

tr = pd.concat([high - low, (high - close.shift()).abs(), (low - close.shift()).abs()], axis=1).max(axis=1)
atr = tr.rolling(14).mean()

plus_di = 100 * (plus_dm.rolling(14).sum() / atr)
minus_di = 100 * (minus_dm.rolling(14).sum() / atr)

dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
data['ADX_14'] = dx.rolling(14).mean()

# On-Balance Volume (OBV)
data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

# Money Flow Index (MFI) - fully vectorized
typical_price = (data['High'] + data['Low'] + data['Close']) / 3
money_flow = typical_price * data['Volume']
tp_diff = typical_price.diff()

positive_mf = money_flow.where(tp_diff > 0, 0).rolling(14).sum()
negative_mf = money_flow.where(tp_diff < 0, 0).rolling(14).sum()
data['MFI_14'] = 100 * (positive_mf / (positive_mf + negative_mf))

# Basic Candlestick Patterns
data['Doji'] = abs(data['Close'] - data['Open']) <= 0.1 * (data['High'] - data['Low'])
data['Hammer'] = ((data['High'] - data['Low']) > 3 * (data['Open'] - data['Close'])) & \
                 ((data['Close'] - data['Low']) / (0.001 + data['High'] - data['Low']) > 0.6)
data['Engulfing'] = ((data['Close'] > data['Open'].shift()) &
                     (data['Open'] < data['Close'].shift()) &
                     (data['Close'] - data['Open'] > data['Open'].shift() - data['Close'].shift()))

In [21]:
# --- 1. Dividends ---
dividends = dividends.reindex(data.index, method='ffill').fillna(0)
data['Dividend'] = dividends

# # --- 2. Splits ---
# splits = splits.reindex(data.index, method='ffill').fillna(0)
# data['Split'] = splits

# # --- 3. Analyst Ratings Changes ---
# # Upgrade/downgrade info
# recs = stock.recommendations
# if recs is not None:
#     recs.index = pd.to_datetime(recs.index)
#     # Flag days with upgrades/downgrades
#     recs['Upgrade'] = recs['To Grade'].apply(lambda x: 1 if x in ['Buy', 'Outperform'] else 0)
#     recs['Downgrade'] = recs['To Grade'].apply(lambda x: 1 if x in ['Underperform', 'Sell'] else 0)
#     upgrades = recs['Upgrade'].reindex(data.index, method='ffill').fillna(0)
#     downgrades = recs['Downgrade'].reindex(data.index, method='ffill').fillna(0)
#     data['Upgrade'] = upgrades
#     data['Downgrade'] = downgrades
# else:
#     data['Upgrade'] = 0
#     data['Downgrade'] = 0

# # --- 4. Earnings Announcements ---
# earnings_dates = stock.earnings_dates
# if earnings_dates is not None:
#     earnings_dates.index = pd.to_datetime(earnings_dates.index)
#     # Create a flag for earnings day
#     data['Earnings'] = 0
#     for date in earnings_dates.index:
#         if date in data.index:
#             data.loc[date, 'Earnings'] = 1
# else:
#     data['Earnings'] = 0
#
# # --- 5. News Headlines (optional sentiment) ---
# news = stock.news
# # Convert news to daily count
# news_dates = [pd.to_datetime(item['providerPublishTime'], unit='s').date() for item in news]
# news_series = pd.Series(1, index=pd.to_datetime(news_dates))
# news_series = news_series.groupby(news_series.index).sum()
# data['NewsCount'] = news_series.reindex(data.index, method='ffill').fillna(0)
#
# # --- Optional: EPS surprises, buybacks from Financial Modeling Prep (requires API key) ---
# # You can fetch via FMP and merge similarly to earnings/dividends

data.tail()


,Close,High,Low,Open,Volume,Return,Volatility_20,Volatility_50,TR,ATR_14,...,MACD,MACD_signal,Stochastic,ADX_14,OBV,MFI_14,Doji,Hammer,Engulfing,Dividend
Price,,,,,,,,,,,,,,,,,,,,,
2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700.0,-0.004032,9.687960,14.471440,2.450012,5.241432,...,7.548993,6.324068,89.984163,27.225712,1.636864e+11,76.965186,True,True,True,0.26
2025-09-30,254.630005,255.919998,253.110001,254.860001,37704300.0,0.000786,9.643641,14.694921,2.809998,4.842146,...,7.496484,6.558551,90.403877,24.874888,1.637241e+11,85.008244,True,False,False,0.26
2025-10-01,255.449997,258.790009,254.929993,255.039993,48713900.0,0.003220,9.954723,14.939439,4.160004,4.867860,...,7.435327,6.733906,88.780614,22.411863,1.637728e+11,85.078627,False,False,False,0.26
2025-10-02,257.130005,258.179993,254.149994,256.579987,42630200.0,0.006577,10.319350,15.183616,4.029999,4.763575,...,7.436696,6.874464,93.013455,20.445273,1.638154e+11,84.932340,False,True,False,0.26
2025-10-03,258.019989,259.239990,253.949997,254.669998,49107000.0,0.003461,10.646582,15.394036,5.289993,4.847146,...,7.424016,6.984375,94.677129,18.323960,1.638645e+11,85.112000,False,True,True,0.26
